In [1]:
import os

In [2]:
import shutil

In [3]:
import subprocess

In [4]:
from snakemake.cli import (
    parse_args, 
    args_to_api, 
    get_argument_parser, 
    _get_executor_plugin_registry, 
    _get_report_plugin_registry, 
    StoragePluginRegistry,
    setup_log_handlers,
    parse_edit_notebook,
    parse_wait_for_files,
    SnakemakeApi,
    OutputSettings,
    DAGSettings,
    StorageSettings,
    DeploymentMethod,
    DeploymentSettings,
    ResourceSettings,
    ConfigSettings,
    parse_config,
    WorkflowSettings,
    PreemptibleRules,
    ExecutionSettings,
    RemoteExecutionSettings,
    SchedulingSettings,
    GroupSettings,
)

In [5]:
config_yaml = """\
x: 1
y: 2
"""

In [6]:
snakefile = """\
configfile: "./config.yaml"

rule calculate_z:
    output:
        "z.txt"
    shell:
        "echo $(({config[x]} + {config[y]})) > z.txt"

rule calculate_w:
    input:
        rules.calculate_z.output
    output:
        "w.txt"
    shell:
        "z=$(cat {input}); echo $(({config[x]} + {config[y]} + $z)) > w.txt"
"""

In [7]:
with open("config.yaml", "w") as f:
    f.writelines(config_yaml)

In [8]:
with open("Snakefile", "w") as f:
    f.writelines(snakefile)

In [9]:
command_str = "snakemake --cores 1 --configfile config.yaml Snakefile -n -p w.txt"

In [10]:
command_str.split(" ")

['snakemake',
 '--cores',
 '1',
 '--configfile',
 'config.yaml',
 'Snakefile',
 '-n',
 '-p',
 'w.txt']

In [11]:
parser = get_argument_parser()
args = parser.parse_args(command_str.split(" ")[1:])

In [12]:
args

Namespace(targets=['w.txt'], dryrun=True, profile=None, workflow_profile=None, cache=None, snakefile=None, cores=1, jobs=None, local_cores=4, resources={}, set_threads={}, max_threads=None, set_resources={}, set_scatter={}, set_resource_scopes={}, default_resources=None, preemptible_rules=None, preemptible_retries=None, configfile=[PosixPath('config.yaml'), PosixPath('Snakefile')], config=None, envvars=set(), directory=None, touch=False, keep_going=False, rerun_triggers=frozenset({<RerunTrigger.MTIME: 0>, <RerunTrigger.INPUT: 2>, <RerunTrigger.SOFTWARE_ENV: 3>, <RerunTrigger.PARAMS: 1>, <RerunTrigger.CODE: 4>}), force=False, executor=None, forceall=False, forcerun=set(), prioritize=set(), batch=None, until=set(), omit_from=set(), rerun_incomplete=False, shadow_prefix=None, scheduler='ilp', wms_monitor=None, wms_monitor_arg=None, scheduler_ilp_solver='COIN_CMD', scheduler_solver_path=None, conda_base_path=None, no_subworkflows=False, precommand=None, groups={}, group_components={}, repo

In [13]:
if args.dryrun:
    args.executor = "dryrun"
elif args.touch:
    args.executor = "touch"
elif args.executor is None:
    args.executor = "local"

if args.report:
    args.reporter = "html"
    args.report_html_path = args.report
    args.report_html_stylesheet_path = args.report_stylesheet

executor_plugin = _get_executor_plugin_registry().get_plugin(args.executor)
executor_settings = executor_plugin.get_settings(args)


In [14]:
storage_provider_settings = {
    name: StoragePluginRegistry().get_plugin(name).get_settings(args)
    for name in StoragePluginRegistry().get_registered_plugins()
}

In [15]:
if args.reporter:
    report_plugin = _get_report_plugin_registry().get_plugin(args.reporter)
    report_settings = report_plugin.get_settings(args)
else:
    report_plugin = None
    report_settings = None

In [16]:
if args.cores is None:
    if executor_plugin.common_settings.local_exec:
        # use --jobs as an alias for --cores
        args.cores = args.jobs
        args.jobs = None
    elif executor_plugin.common_settings.dryrun_exec:
        args.cores = 1
        args.jobs = None

In [17]:
# start profiler if requested
if args.runtime_profile:
    import yappi

    yappi.start()

In [18]:
log_handlers = setup_log_handlers(args, parser)
edit_notebook = parse_edit_notebook(args)
wait_for_files = parse_wait_for_files(args)

In [19]:
snakemake_api = SnakemakeApi(
    OutputSettings(
        printshellcmds=args.printshellcmds,
        nocolor=args.nocolor,
        quiet=args.quiet,
        debug_dag=args.debug_dag,
        verbose=args.verbose,
        show_failed_logs=args.show_failed_logs,
        log_handlers=log_handlers,
        keep_logger=False,
        stdout=args.dryrun,
        benchmark_extended=args.benchmark_extended,
    )
) 

In [20]:
deployment_method = args.software_deployment_method
if args.use_conda:
    deployment_method.add(DeploymentMethod.CONDA)
if args.use_apptainer:
    deployment_method.add(DeploymentMethod.APPTAINER)
if args.use_envmodules:
    deployment_method.add(DeploymentMethod.ENV_MODULES)


In [21]:
storage_settings = StorageSettings(
    default_storage_provider=args.default_storage_provider,
    default_storage_prefix=args.default_storage_prefix,
    local_storage_prefix=args.local_storage_prefix,
    remote_job_local_storage_prefix=args.remote_job_local_storage_prefix,
    shared_fs_usage=args.shared_fs_usage,
    keep_storage_local=args.keep_storage_local_copies,
    notemp=args.notemp,
    all_temp=args.all_temp,
    unneeded_temp_files=args.unneeded_temp_files,
)


In [22]:
snakemake_api._is_in_context

False

In [23]:
snakemake_api._is_in_context = True

In [24]:
workflow_api = snakemake_api.workflow(
    resource_settings=ResourceSettings(
        cores=args.cores,
        nodes=args.jobs,
        local_cores=args.local_cores,
        max_threads=args.max_threads,
        resources=args.resources,
        overwrite_threads=args.set_threads,
        overwrite_scatter=args.set_scatter,
        overwrite_resource_scopes=args.set_resource_scopes,
        overwrite_resources=args.set_resources,
        default_resources=args.default_resources,
    ),
    config_settings=ConfigSettings(
        config=parse_config(args.config),
        configfiles=args.configfile,
        config_args=args.config,
    ),
    storage_settings=storage_settings,
    storage_provider_settings=storage_provider_settings,
    workflow_settings=WorkflowSettings(
        wrapper_prefix=args.wrapper_prefix,
        exec_mode=args.mode,
        cache=args.cache,
    ),
    deployment_settings=DeploymentSettings(
        deployment_method=deployment_method,
        conda_prefix=args.conda_prefix,
        conda_cleanup_pkgs=args.conda_cleanup_pkgs,
        conda_base_path=args.conda_base_path,
        conda_frontend=args.conda_frontend,
        conda_not_block_search_path_envvars=args.conda_not_block_search_path_envvars,
        apptainer_args=args.apptainer_args,
        apptainer_prefix=args.apptainer_prefix,
    ),
    snakefile=args.snakefile,
    workdir=args.directory,
)

In [25]:
dag = workflow_api.dag()

Config file ./config.yaml is extended by additional config specified via the command line.


In [26]:
dag.workflow_api._workflow.config

{'x': 1,
 'y': 2,
 'configfile': './config.yaml',
 'rule calculate_z': {'output': 'z.txt',
  'shell': 'echo $(({config[x]} + {config[y]})) > z.txt'},
 'rule calculate_w': {'input': 'rules.calculate_z.output',
  'output': 'w.txt',
  'shell': 'z=$(cat {input}); echo $(({config[x]} + {config[y]} + $z)) > w.txt'}}